In [2]:
import tensorflow as tf

from src.swin_transformer.model import *
from src.swin_transformer.helper import *
from src.swin_transformer.patches import *
from src.swin_transformer.window_attn import *


In [3]:
model = SwinTransformer(**cfg)

2022-05-24 15:11:42.837443: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 15:11:42.838888: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 15:11:42.844515: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 15:11:42.845324: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 15:11:42.846072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [5]:
model.build([2,384,384,3])

In [6]:
model.summary()

Model: "swin_transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 projection (Sequential)     multiple                  6528      
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 basic_layer (BasicLayer)    multiple                  532872    
                                                                 
 basic_layer_1 (BasicLayer)  multiple                  2114320   
                                                                 
 basic_layer_2 (BasicLayer)  multiple                  58996512  
                                                                 
 basic_layer_3 (BasicLayer)  multiple                  25226304  
                                                                 
 layer_normalization_52 (Lay  multiple            